# VacationPy
---

## Import Libraries and Load the Weather and Coordinates Data

In [4]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages (getting unnecessary warnings about blowfish being depracated for map generations)
import warnings
warnings.filterwarnings("ignore")

# Import API key
from api_keys import geoapify_key

In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,leer,53.2316,7.4610,14.41,94,13,3.14,DE,1690255348
1,1,isabela,6.7085,121.9711,30.03,69,83,6.06,PH,1690255348
2,2,kununurra,-15.7667,128.7333,20.99,24,10,7.20,AU,1690255349
3,3,nosivka,50.9315,31.5828,17.75,63,87,4.18,UA,1690255349
4,4,hawaiian paradise park,19.5933,-154.9731,26.73,89,100,3.09,US,1690255349


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:
# %%capture --no-display

map_plot_city = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo= True,
    tiles = "OSM",
    size="Humidity",
    color = "City",
    frame_width =800,
    frame_height =600,
    alpha= 0.5
    )

# Display the map
display(map_plot_city)

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_city= city_data_df[(city_data_df["Max Temp"]>20) & (city_data_df["Max Temp"]<25) & (city_data_df["Cloudiness"]==0) & (city_data_df["Wind Speed"]<4.5)]

# Drop any rows with null values
ideal_weather_city=ideal_weather_city.dropna()

# Display sample data
ideal_weather_city

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
97,97,bethel,41.3712,-73.4140,24.20,80,0,1.54,US,1690255313
131,131,hamilton,39.1834,-84.5333,22.64,88,0,2.06,US,1690255032
141,141,rawlins,41.7911,-107.2387,22.94,33,0,4.12,US,1690255232
173,173,la ronge,55.1001,-105.2842,22.14,60,0,1.03,CA,1690255389
193,193,griffith,41.5284,-87.4237,24.05,77,0,0.00,US,1690255393
291,291,myrina,39.8797,25.0742,22.58,73,0,0.00,GR,1690255237
391,391,miracema do tocantins,-9.5618,-48.3967,23.18,40,0,0.87,BR,1690255453
432,432,brookings,44.3114,-96.7984,24.00,77,0,3.09,US,1690255315
435,435,terenos,-20.4422,-54.8603,21.84,56,0,1.54,BR,1690255464
442,442,quincy,42.2529,-71.0023,23.52,84,0,2.24,US,1690255466


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_city.loc[:,["City","Country", 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]= ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
97,bethel,US,41.3712,-73.4140,80,
131,hamilton,US,39.1834,-84.5333,88,
141,rawlins,US,41.7911,-107.2387,33,
173,la ronge,CA,55.1001,-105.2842,60,
193,griffith,US,41.5284,-87.4237,77,
291,myrina,GR,39.8797,25.0742,73,
391,miracema do tocantins,BR,-9.5618,-48.3967,40,
432,brookings,US,44.3114,-96.7984,77,
435,terenos,BR,-20.4422,-54.8603,56,
442,quincy,US,42.2529,-71.0023,84,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel
radius = 10000
params = {
        "categories": "accommodation.hotel",
        "radius": radius,
        "apiKey": geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")
print("---------------------")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params).json()
 
    # Convert the API response to JSON format
    name_address = requests.get(base_url, params=params).json()
        
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")


Starting hotel search
---------------------
bethel - nearest hotel: Hampton Inn Danbury
hamilton - nearest hotel: North Vista Manor
rawlins - nearest hotel: Quality Inn Rawlins I-80
la ronge - nearest hotel: La Ronge Hotel and Suites
griffith - nearest hotel: Quality Inn & Suites
myrina - nearest hotel: Ήφαιστος
miracema do tocantins - nearest hotel: No hotel found
brookings - nearest hotel: Quality Inn
terenos - nearest hotel: No hotel found
quincy - nearest hotel: Howard Johnson
la rinconada - nearest hotel: Hotel Torre de los Guzmanes
adler - nearest hotel: Shine


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [12]:
# %%capture --no-display

# Configure the map plot
map_plot_city = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo= True,
    tiles = "OSM",
    hover_cols=["Hotel Name", "Country"],
    size="Humidity",
    color = "City",
    frame_width =800,
    frame_height =600,
    alpha= 0.8
    )

# Display the map
display(map_plot_city)


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)